<a href="https://colab.research.google.com/github/Olfeng-xalaz/SCM/blob/main/Spediteur_erweitert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip als Paketmanager
! pip install -q pyscipopt

In [ ]:
from pyscipopt import Model, quicksum

# Optimierungsmodell für Spediteur

In [ ]:
# Erstellen einer Modellinstanz
scip = Model()

# Indexmengen

In [ ]:
I = ['A1','A2','A3','A4','A5','A6',] # Menge der Aufträge

# Entscheidungsvariablen

In [ ]:
x = {} # Nehme ich meinen Auftrag an oder nicht?
for i in I:
  x[i] = scip.addVar(vtype='B', name=f'x_{i}') # Boolean für "ja" oder "nein"
print('Entscheidungsvariablen =', scip.getVars())

Entscheidungsvariablen = [x_A1, x_A2, x_A3, x_A4, x_A5, x_A6]


# Parameter

In [ ]:
db = {} # Deckungsbeitrag der Aufträge
db ['A1'] = 10
db ['A2'] = 15
db ['A3'] = 25
db ['A4'] = 5
db ['A5'] = 11
db ['A6'] = 8

v = {} # Volumen der Aufträge
v ['A1'] = 3
v ['A2'] = 4.5
v ['A3'] = 9.25
v ['A4'] = 1.2
v ['A5'] = 4
v ['A6'] = 2.7

cap = 10 # Kapazität des LKWs

# Zielfunktion

In [ ]:
scip.setObjective(quicksum(db[i] * x[i] for i in I), sense = 'maximize') # Maximierung des GesamtDB

# Restriktionen

In [ ]:
for i in I:
  scip.addCons(quicksum(v[i] * x [i] for i in I) <= cap, name = i) # Restriktion auf Kapazität des LKWs
print("Nebenbedingung: ", scip.getConss())

Nebenbedingung:  [A1, A2, A3, A4, A5, A6]


# Berechnung des Ergebnisses

In [ ]:
scip.optimize()
status = scip.getStatus()
gap = scip.getGap()
print(f"Status des Solvers: {status} , GAP={gap}% \n")

if status == 'optimal':
  print("Lösung:")
  print("Zielfunktionswert (Deckungsbeitrag) = ",round(scip.getObjVal())) # Erzielter maximaler Deckungsbeitrag bei optimaler Verteilung der Aufträge
  for i in I:
    print(f'{i} = ',round(scip.getVal(x[i]))) # Welche LKWs sollte man nehmen?
else:
  print("Keine Lösung gefunden!!")


Status des Solvers: optimal , GAP=0.0% 

Lösung:
Zielfunktionswert (Deckungsbeitrag) =  31
A1 =  0
A2 =  1
A3 =  0
A4 =  1
A5 =  1
A6 =  0
